In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
# week_nums = [190921] 
week_nums = [190921, 190914, 190907, 190831, 190824, 190817, 190810, 190803, 190727, 190720, 190713, 190706]
turnstiles_df = get_data(week_nums)

In [6]:
turnstiles_df.head()

,Unnamed: 0,C/A,UNIT,SCP,STATION,LINENAME,DATE,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,DIFF_ENTRIES,DIFF_EXITS
0,1,A002,R051,02-00-00,59 ST,NQR456W,06/30/2019,7117480,2410406,06/29/2019,7116734.0,2410183.0,746.0,223.0
1,2,A002,R051,02-00-00,59 ST,NQR456W,07/01/2019,7118119,2410590,06/30/2019,7117480.0,2410406.0,639.0,184.0
2,3,A002,R051,02-00-00,59 ST,NQR456W,07/02/2019,7119610,2410993,07/01/2019,7118119.0,2410590.0,1491.0,403.0
3,4,A002,R051,02-00-00,59 ST,NQR456W,07/03/2019,7121181,2411468,07/02/2019,7119610.0,2410993.0,1571.0,475.0
4,5,A002,R051,02-00-00,59 ST,NQR456W,07/04/2019,7122652,2411887,07/03/2019,7121181.0,2411468.0,1471.0,419.0


In [4]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]

In [5]:
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [6]:
# counts for a week of data
turnstiles_df.DATE.value_counts().sort_index()

06/29/2019    29239
06/30/2019    29130
07/01/2019    29306
07/02/2019    30114
07/03/2019    29965
              ...  
09/16/2019    29174
09/17/2019    29602
09/18/2019    29232
09/19/2019    29262
09/20/2019    29261
Name: DATE, Length: 84, dtype: int64

In [7]:
from datetime import datetime as dt

In [8]:
# Take the date and time fields into a single datetime column
turnstiles_df['DATE_TIME'] = pd.to_datetime(turnstiles_df['DATE'] + ' ' + turnstiles_df['TIME'], 
                                            format="%m/%d/%Y %H:%M:%S")

In [9]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,00:00:00,REGULAR,7198818,2438323,2019-09-14 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,04:00:00,REGULAR,7198834,2438325,2019-09-14 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,08:00:00,REGULAR,7198847,2438354,2019-09-14 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,12:00:00,REGULAR,7198929,2438428,2019-09-14 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,16:00:00,REGULAR,7199125,2438483,2019-09-14 16:00:00


In [10]:
# Drop Exits and Desc Column.  To prevent errors in multiple run of cell, errors on drop is ignored
turnstiles_df = turnstiles_df.drop(['DESC', 'DIVISION'], axis=1, errors="ignore")

In [11]:
turnstiles_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,TIME,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,09/14/2019,00:00:00,7198818,2438323,2019-09-14 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,09/14/2019,04:00:00,7198834,2438325,2019-09-14 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,09/14/2019,08:00:00,7198847,2438354,2019-09-14 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,09/14/2019,12:00:00,7198929,2438428,2019-09-14 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,09/14/2019,16:00:00,7199125,2438483,2019-09-14 16:00:00


In [12]:
turnstiles_daily = (turnstiles_df
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "LINENAME", "DATE"],as_index=False)
                        ['ENTRIES', 'EXITS'].first())
turnstiles_hourly = (turnstiles_df
                        .groupby(["C/A", "UNIT", "SCP", "STATION", "LINENAME", "DATE_TIME"],as_index=False)
                        ['ENTRIES', 'EXITS'].first())

In [13]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,06/29/2019,7116734,2410183
1,A002,R051,02-00-00,59 ST,NQR456W,06/30/2019,7117480,2410406
2,A002,R051,02-00-00,59 ST,NQR456W,07/01/2019,7118119,2410590
3,A002,R051,02-00-00,59 ST,NQR456W,07/02/2019,7119610,2410993
4,A002,R051,02-00-00,59 ST,NQR456W,07/03/2019,7121181,2411468


In [14]:
turnstiles_hourly.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE_TIME,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 00:00:00,7116734,2410183
1,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 08:00:00,7116770,2410218
2,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 12:00:00,7116856,2410288
3,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 16:00:00,7117076,2410346
4,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 20:00:00,7117340,2410387


In [15]:
turnstiles_daily[['PREV_DATE', 'PREV_ENTRIES', 'PREV_EXITS']] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES", 'EXITS']
                                                       .apply(lambda grp: grp.shift(1)))
turnstiles_hourly[['PREV_DATE', 'PREV_ENTRIES', 'PREV_EXITS']] = (turnstiles_hourly
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE_TIME", "ENTRIES", 'EXITS']
                                                       .apply(lambda grp: grp.shift(1)))

In [16]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,06/29/2019,7116734,2410183,NaN,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,06/30/2019,7117480,2410406,06/29/2019,7116734.0,2410183.0
2,A002,R051,02-00-00,59 ST,NQR456W,07/01/2019,7118119,2410590,06/30/2019,7117480.0,2410406.0
3,A002,R051,02-00-00,59 ST,NQR456W,07/02/2019,7119610,2410993,07/01/2019,7118119.0,2410590.0
4,A002,R051,02-00-00,59 ST,NQR456W,07/03/2019,7121181,2411468,07/02/2019,7119610.0,2410993.0


In [17]:
turnstiles_hourly.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE_TIME,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 00:00:00,7116734,2410183,NaT,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 08:00:00,7116770,2410218,2019-06-29 00:00:00,7116734.0,2410183.0
2,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 12:00:00,7116856,2410288,2019-06-29 08:00:00,7116770.0,2410218.0
3,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 16:00:00,7117076,2410346,2019-06-29 12:00:00,7116856.0,2410288.0
4,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 20:00:00,7117340,2410387,2019-06-29 16:00:00,7117076.0,2410346.0


In [18]:
# Drop the rows for the earliest date in the df
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
turnstiles_hourly.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [19]:
def get_entry_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_daily["DIFF_ENTRIES"] = turnstiles_daily.apply(get_entry_counts, axis=1, max_counter=1000000)
turnstiles_hourly["DIFF_ENTRIES"] = turnstiles_hourly.apply(get_entry_counts, axis=1, max_counter=1000000)

12 6170094.0
824 3557655.0
458818 12876202.0
459073 68448066.0
143 1599329.0
2000091722 6897017.0
460787 6805255.0
11 2029641496.0
73 6697765.0
7 2997591.0
5327464 4320256.0
4320257 2920966.0
875479593 775219651.0
590840 9156730.0
319 5420925.0
134 1478959.0
721090 5343128.0
222 6053291.0
262 3407291.0
26 1652502.0
422 17135909.0
22 2129136933.0
21 13593836.0
27 7808518.0
21 2129136817.0
28 13594033.0
21 7809012.0
80 2781090.0
304 4587522.0
1962917508 9137626.0
393224 117441229.0
23 2130567.0
721441481 2880437.0
352361657 1409610.0
14 4584481.0
80 3260287.0
134 3376441.0
351 3839369.0
1487 4721413.0
983271 3670866.0
171 1788082.0
1325342695 11664583.0
1507350 3900.0
535 4777158.0
102459236 210849.0
105 7542484.0
458765 6627959.0
354 5366386.0
524295 2338090.0
1359886 4065.0
70 5631392.0
871 2543403.0
892 1975856.0
37 1514083866.0
34 12122524.0
25 1229078732.0
1572864 2139.0
2473 22554699.0
1233 4018083.0
214 9789350.0
281 1449982.0
7 1370664.0
2162762 33622258.0
592416496 389431.0
6 57

In [20]:
def get_exit_counts(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["EXITS"], row["PREV_EXITS"])
        counter = min(row["EXITS"], row["PREV_EXITS"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_daily["DIFF_EXITS"] = turnstiles_daily.apply(get_exit_counts, axis=1, max_counter=1000000)
turnstiles_hourly["DIFF_EXITS"] = turnstiles_hourly.apply(get_exit_counts, axis=1, max_counter=1000000)

8 20876715.0
1314 1109237.0
526 1833014.0
43 2255995.0
166 16906899.0
0 1045674.0
59 1109387.0
1850345261 8702864.0
155 1010120.0
3 702629110.0
147 3570100.0
0 4745858.0
3430095 8440580.0
4745860 2455688.0
8440589 1962487.0
996500290 1698414914.0
71 3688130.0
219 10787335.0
382 1298864.0
327 6368810.0
435 4923537.0
451 2579110.0
205 10179613.0
16 1155490186.0
14 15837624.0
15 4262163.0
17 1155489758.0
12 15838154.0
14 4262925.0
42 1398489.0
317 3351226.0
318790205 1305671.0
1895801237 9396077.0
604040484 2397891.0
1 1161875.0
435 12613231.0
1137 10021370.0
165 1692611.0
687875257 2749154.0
1060 13388806.0
1431462 322352.0
95 3163183.0
1908 13790712.0
910 3642394.0
144 3425513.0
712 1437147.0
120 12292134.0
379 2592663.0
17 908660197.0
14 7483993.0
15 1921803310.0
487 5600551.0
671 1919118.0
1112 12197572.0
453 219039396.0
211 1435950.0
886336027 202295.0
2467 2526158.0
3704 13303075.0
3331 616256155.0
569320805 701522.0
65993790 301732246.0
423 505935940.0
20003723 822184273.0
14093393

In [21]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,DIFF_ENTRIES,DIFF_EXITS
1,A002,R051,02-00-00,59 ST,NQR456W,06/30/2019,7117480,2410406,06/29/2019,7116734.0,2410183.0,746.0,223.0
2,A002,R051,02-00-00,59 ST,NQR456W,07/01/2019,7118119,2410590,06/30/2019,7117480.0,2410406.0,639.0,184.0
3,A002,R051,02-00-00,59 ST,NQR456W,07/02/2019,7119610,2410993,07/01/2019,7118119.0,2410590.0,1491.0,403.0
4,A002,R051,02-00-00,59 ST,NQR456W,07/03/2019,7121181,2411468,07/02/2019,7119610.0,2410993.0,1571.0,475.0
5,A002,R051,02-00-00,59 ST,NQR456W,07/04/2019,7122652,2411887,07/03/2019,7121181.0,2411468.0,1471.0,419.0


In [22]:
turnstiles_hourly.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE_TIME,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,DIFF_ENTRIES,DIFF_EXITS
1,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 08:00:00,7116770,2410218,2019-06-29 00:00:00,7116734.0,2410183.0,36.0,35.0
2,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 12:00:00,7116856,2410288,2019-06-29 08:00:00,7116770.0,2410218.0,86.0,70.0
3,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 16:00:00,7117076,2410346,2019-06-29 12:00:00,7116856.0,2410288.0,220.0,58.0
4,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 20:00:00,7117340,2410387,2019-06-29 16:00:00,7117076.0,2410346.0,264.0,41.0
5,A002,R051,02-00-00,59 ST,NQR456W,2019-06-30 00:00:00,7117480,2410406,2019-06-29 20:00:00,7117340.0,2410387.0,140.0,19.0


In [7]:
stops = ['FOURTH AV',
'SEVENTH AV',
'EIGHTH AV',
'NINTH AV',
'NINTH ST',
'15 ST–PROSPECT PARK',
'18 AV',
'20 AV',
'25 AV',
'45 ST',
'53 ST',
'55 ST',
'62 ST',
'71 ST',
'ALABAMA AV',
'ATLANTIC AV',
'ATLANTIC AV–BARCLAYS CENTER',
'AV H',
'AV I',
'AV J',
'AV M',
'AV N',
'AV P',
'AV U',
'AV X',
'BAY 50 ST',
'BAY PKWY',
'BAY RIDGE AV',
'BAY RIDGE–95 ST',
'BEDFORD AV',
'BEDFORD–NOSTRAND AVS',
'BERGEN ST',
'BEVERLEY ROAD',
'BEVERLY ROAD',
'BOROUGH HALL',
'BRIGHTON BEACH',
'BROADWAY JUNCTION',
'BUSHWICK AV–ABERDEEN ST',
'CANARSIE–ROCKAWAY PKWY',
'CARROLL ST',
'CENTRAL AV',
'CHAUNCEY ST',
'CHURCH AV',
'CLARK ST',
'CLASSON AV',
'CLEVELAND ST',
'CLINTON–WASHINGTON AVS',
'CONEY ISLAND–STILLWELL AV',
'CORTELYOU ROAD',
'COURT ST',
'CRESCENT ST',
'CROWN HEIGHTS–UTICA AV',
'CYPRESS HILLS',
'DEKALB AV',
'DITMAS AV',
'EAST 105TH ST',
'EASTERN PKWY–BROOKLYN MUSEUM',
'EUCLID AV',
'FLATBUSH AV–BROOKLYN COLLEGE†',
'FLUSHING AV',
'FORT HAMILTON PKWY',
'FRANKLIN AV',
'GATES AV',
'GRAHAM AV',
'GRAND ARMY PLAZA',
'GRANT AV',
'GREENPOINT AV',
'HALSEY ST',
'HEWES ST',
'HIGH ST',
'HOYT ST',
'HOYT–SCHERMERHORN STS',
'JAY ST–METROTECH',
'JEFFERSON ST',
'JUNIUS ST',
'KINGS HIGHWAY',
'KINGSTON AV',
'KINGSTON–THROOP AVS',
'KNICKERBOCKER AV',
'KOSCIUSZKO ST',
'LAFAYETTE AV',
'LIBERTY AV',
'LIVONIA AV',
'LORIMER ST',
'MARCY AV',
'MONTROSE AV',
'MORGAN AV',
'MYRTLE AV',
'MYRTLE–WILLOUGHBY AVS',
'MYRTLE–WYCKOFF AVS',
'MYRTLE–WYCKOFF AVS',
'NASSAU AV',
'NECK ROAD',
'NEPTUNE AV',
'NEVINS ST',
'NEW UTRECHT AV',
'NEWKIRK AV',
'NEWKIRK PLAZA',
'NORWOOD AV',
'NOSTRAND AV',
'OCEAN PKWY',
'PARKSIDE AV',
'PENNSYLVANIA AV',
'PRESIDENT ST',
'PROSPECT AV',
'PROSPECT PARK',
'RALPH AV',
'ROCKAWAY AV',
'SARATOGA AV',
'SHEEPSHEAD BAY',
'SHEPHERD AV',
'SMITH–NINTH STS',
'STERLING ST',
'SUTTER AV',
'SUTTER AV–RUTLAND ROAD',
'UNION ST',
'UTICA AV',
'VAN SICLEN AV',
'WEST EIGHTH ST–NEW YORK AQUARIUM',
'WILSON AV',
'WINTHROP ST',
'YORK ST']

In [9]:
turnstiles_daily = turnstiles_daily[turnstiles_daily['STATION'].isin(stops)]
turnstiles_daily

,Unnamed: 0,C/A,UNIT,SCP,STATION,LINENAME,DATE,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,DIFF_ENTRIES,DIFF_EXITS
2988,27358,B012,R196,00-00-00,PROSPECT PARK,BQS,06/30/2019,1568780299,786494205,06/29/2019,1.568779e+09,786495477.0,1101.0,1272.0
2989,27359,B012,R196,00-00-00,PROSPECT PARK,BQS,07/01/2019,1568781387,786492865,06/30/2019,1.568780e+09,786494205.0,1088.0,1340.0
2990,27360,B012,R196,00-00-00,PROSPECT PARK,BQS,07/02/2019,1568783051,786491255,07/01/2019,1.568781e+09,786492865.0,1664.0,1610.0
2991,27361,B012,R196,00-00-00,PROSPECT PARK,BQS,07/03/2019,1568784512,786489750,07/02/2019,1.568783e+09,786491255.0,1461.0,1505.0
2992,27362,B012,R196,00-00-00,PROSPECT PARK,BQS,07/04/2019,1568786107,786488123,07/03/2019,1.568785e+09,786489750.0,1595.0,1627.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80561,403338,R644,R135,01-06-01,NEWKIRK AV,25,09/16/2019,578339,10167482,09/15/2019,5.783380e+05,10167480.0,1.0,2.0
80562,403339,R644,R135,01-06-01,NEWKIRK AV,25,09/17/2019,578431,10169294,09/16/2019,5.783390e+05,10167482.0,92.0,1812.0
80563,403340,R644,R135,01-06-01,NEWKIRK AV,25,09/18/2019,578507,10171264,09/17/2019,5.784310e+05,10169294.0,76.0,1970.0
80564,403341,R644,R135,01-06-01,NEWKIRK AV,25,09/19/2019,578598,10173259,09/18/2019,5.785070e+05,10171264.0,91.0,1995.0


In [33]:
turnstiles_hourly = turnstiles_hourly[turnstiles_hourly['STATION'].isin(stops)]
turnstiles_hourly

,C/A,UNIT,SCP,STATION,LINENAME,DATE_TIME,ENTRIES,EXITS,PREV_DATE,PREV_ENTRIES,PREV_EXITS,DIFF_ENTRIES,DIFF_EXITS
1,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 08:00:00,7116770,2410218,2019-06-29 00:00:00,7116734.0,2410183.0,36.0,35.0
2,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 12:00:00,7116856,2410288,2019-06-29 08:00:00,7116770.0,2410218.0,86.0,70.0
3,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 16:00:00,7117076,2410346,2019-06-29 12:00:00,7116856.0,2410288.0,220.0,58.0
4,A002,R051,02-00-00,59 ST,NQR456W,2019-06-29 20:00:00,7117340,2410387,2019-06-29 16:00:00,7117076.0,2410346.0,264.0,41.0
5,A002,R051,02-00-00,59 ST,NQR456W,2019-06-30 00:00:00,7117480,2410406,2019-06-29 20:00:00,7117340.0,2410387.0,140.0,19.0


In [10]:
#print(turnstiles_daily['STATION'].unique())
turnstiles_daily.to_csv('turnstiles_daily_filtered.csv')
turnstiles_hourly.to_csv('turnstiles_hourly_filtered.csv')

In [35]:
print(max(turnstiles_daily['DIFF_ENTRIES']))
print(max(turnstiles_daily['DIFF_EXITS']))

915263.0
374281.0


In [36]:
print(max(turnstiles_hourly['DIFF_ENTRIES']))
print(max(turnstiles_hourly['DIFF_EXITS']))

915984.0
381308.0
